# Workshop: Olympic Medals

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/Olympic_rings_without_rims.svg/1920px-Olympic_rings_without_rims.png" width=300>

First clone the repository at [https://github.com/daniprec/BAM-Coding-Lab](https://github.com/daniprec/BAM-Coding-Lab)

You will find the data inside `data/olympics`.


---

### ➕ Adding More Data

Merge both datasets and save the result.

In [ ]:
path_csv = path_folder + "/olympics_medals_2024.csv"

df_new = pd.read_csv(path_csv)
df_new.head(5)

NameError: name 'path_folder' is not defined

In [ ]:
df_new["medal_code"].value_counts()

medal_code
3    381
2    327
1    326
Name: count, dtype: int64